# calculate urban clusters

### Classifies urban clusters using the Degree of urbanisation methodology (https://ghsl.jrc.ec.europa.eu/degurbaDefinitions.php)

This methodology uses consistent 1X1 kilometer grid cells to classify cells based on population density, contiguity, and population size.

In [1]:
# import libraries
import sys, os, inspect, logging, importlib
import rasterio
import pandas as pd
import geopandas as gpd

In [2]:
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
module_path

'C:\\Users\\wb546254\\Documents\\GitHub\\INFRA_SAP'

In [4]:
from infrasap.UrbanRaster import *

## Inputs include a population grid. In this analysis we will calculate urban clusters using both GHS-Pop and WorldPop

In [9]:
# bring in GHS-POP
ghs_pop_ubz_file = r"C:\Users\wb546254\OneDrive - WBG\Documents\ubz_urban_analysis\ghs_pop_UBZ_merged_mollweide.tif"

In [10]:
# checkout how GHS-POP is in Mollweide projection
# This means that each pixel represents a consistent 1X1 kilometer area
ghs_pop_ubz = rasterio.open(ghs_pop_ubz_file)
ghs_pop_ubz.crs

CRS.from_wkt('PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [11]:
# bring in WorldPop
worldpop_ubz_file = r"C:\Users\wb546254\OneDrive - WBG\Documents\ubz_urban_analysis\uzb_ppp_2020_1km_Aggregated.tif"

WorldPop is an unprojected WGS84 raster. The strategy we will use is to calculate the area in sq km of each pixel in the  raster. This will enable us to create a normalized version of the raster that contains population count per sq km for the pixel values. 

In [12]:
# The WorldPop file is unprojected in WGS84
# Each pixel is 0.00833333 by 0.00833333 decimal degrees, which is approximately 1X1 kilometer at the equator
# The problem with unprojected WGS84 data is that the pixel area changes depending on the latitude.
worldpop_ubz = rasterio.open(worldpop_ubz_file)
worldpop_ubz.crs

CRS.from_epsg(4326)

In [13]:
# calc_areagrid calculates the area in sq km of each pixel
worldpop_UBZ_areagrid = calc_areagrid(worldpop_ubz_file)
worldpop_UBZ_areagrid

array([[0.60249174, 0.60249174, 0.60249174, ..., 0.60249174, 0.60249174,
        0.60249174],
       [0.60257994, 0.60257994, 0.60257994, ..., 0.60257994, 0.60257994,
        0.60257994],
       [0.60266814, 0.60266814, 0.60266814, ..., 0.60266814, 0.60266814,
        0.60266814],
       ...,
       [0.68418225, 0.68418225, 0.68418225, ..., 0.68418225, 0.68418225,
        0.68418225],
       [0.68425649, 0.68425649, 0.68425649, ..., 0.68425649, 0.68425649,
        0.68425649],
       [0.68433073, 0.68433073, 0.68433073, ..., 0.68433073, 0.68433073,
        0.68433073]])

In [14]:
# convert to float32
print(worldpop_UBZ_areagrid.dtype)
worldpop_UBZ_areagrid = np.float32(worldpop_UBZ_areagrid)
print(worldpop_UBZ_areagrid.dtype)

float64
float32


In [15]:
# create numpy ndarray of worldpop_ubz
# pop_values
with rasterio.open(worldpop_ubz_file, 'r') as ds:
    worldpop_ubz_ndarray = ds.read()  # read all raster values

In [16]:
ds.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -99999.0,
 'width': 2056,
 'height': 1007,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.0083333333, 0.0, 55.99874978222229,
        0.0, -0.0083333333, 45.574583339123905)}

In [17]:
type(worldpop_ubz_ndarray)

numpy.ndarray

In [18]:
worldpop_ubz_ndarray

array([[[-99999., -99999., -99999., ..., -99999., -99999., -99999.],
        [-99999., -99999., -99999., ..., -99999., -99999., -99999.],
        [-99999., -99999., -99999., ..., -99999., -99999., -99999.],
        ...,
        [-99999., -99999., -99999., ..., -99999., -99999., -99999.],
        [-99999., -99999., -99999., ..., -99999., -99999., -99999.],
        [-99999., -99999., -99999., ..., -99999., -99999., -99999.]]],
      dtype=float32)

In [19]:
worldpop_ubz_ndarray.shape

(1, 1007, 2056)

### flatten array to 2 dimensions

In [20]:
# One shape dimension can be -1. 
# In this case, the value is inferred from 
# the length of the array and remaining dimensions.
worldpop_ubz_ndarray_2d = worldpop_ubz_ndarray.reshape(-1, worldpop_ubz_ndarray.shape[-1])

In [21]:
worldpop_ubz_ndarray_2d.shape

(1007, 2056)

In [22]:
# create a new array where values are pop_values / sq km
# pop_values may contain -99999 values for nodata, in this case the new values will still be large negative values.
# This should be OK
# new_array = pop_values / areagrid
pop_per_sq_km = np.divide(worldpop_ubz_ndarray_2d, worldpop_UBZ_areagrid)

In [23]:
pop_per_sq_km

array([[-165975.72, -165975.72, -165975.72, ..., -165975.72, -165975.72,
        -165975.72],
       [-165951.42, -165951.42, -165951.42, ..., -165951.42, -165951.42,
        -165951.42],
       [-165927.12, -165927.12, -165927.12, ..., -165927.12, -165927.12,
        -165927.12],
       ...,
       [-146158.44, -146158.44, -146158.44, ..., -146158.44, -146158.44,
        -146158.44],
       [-146142.56, -146142.56, -146142.56, ..., -146142.56, -146142.56,
        -146142.56],
       [-146126.72, -146126.72, -146126.72, ..., -146126.72, -146126.72,
        -146126.72]], dtype=float32)

### set all negative values to -99999.0, which is the NoData value

In [24]:
pop_per_sq_km[pop_per_sq_km < 0] = -99999.0

In [25]:
with rasterio.open(worldpop_ubz_file, 'r') as ds:
    print(ds.meta)
    ulX = ds.meta['transform'][2]
    ulY = ds.meta['transform'][5]

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -99999.0, 'width': 2056, 'height': 1007, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0083333333, 0.0, 55.99874978222229,
       0.0, -0.0083333333, 45.574583339123905)}


## Save ndarray as raster

In [26]:
# https://gis.stackexchange.com/questions/279953/numpy-array-to-gtiff-using-rasterio-without-source-raster

from rasterio.transform import from_origin

#arr = np.random.randint(5, size=(100,100)).astype(np.float)

# rasterio.transform.from_origin(west, north, xsize, ysize)
# Return an Affine transformation given upper left and pixel sizes.
transform = from_origin(ulX,ulY,0.00833333,0.00833333)

new_dataset = rasterio.open('normalized_worldpop_per_km_sq.tif', 'w', driver='GTiff',
                            height = pop_per_sq_km.shape[0], width = pop_per_sq_km.shape[1],
                            count=1, dtype=str(pop_per_sq_km.dtype),
                            crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs',
                            transform=transform)

new_dataset.nodata = -99999.0
new_dataset.write(pop_per_sq_km, 1)
new_dataset.close()

In [27]:
# inspect
with rasterio.open('normalized_worldpop_per_km_sq.tif', 'r') as ds:
    print(ds.meta)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -99999.0, 'width': 2056, 'height': 1007, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00833333, 0.0, 55.99874978222229,
       0.0, -0.00833333, 45.574583339123905)}


## Calculate Urban Extents

### run normalized_world_pop urban centres

In [28]:
# import normalized WorldPoP file
normalized_world_pop = r"C:\Users\wb546254\Documents\GitHub\INFRA_SAP\Notebooks\Implementations\normalized_worldpop_per_km_sq.tif"

In [29]:
urban_raster = urbanGriddedPop(normalized_world_pop)

In [30]:
urban_extents = urban_raster.calculateUrban(densVal=1500, totalPopThresh=50000, smooth=True)

In [31]:
wp_urbanExtents = r"UBZ_urban_centres_world_pop_normalized_smooth.shp"

In [32]:
#urban_extents.to_csv(wp_urbanExtents)
urban_extents.to_file(wp_urbanExtents)

### run normalized_world_pop urban clusters

In [33]:
urban_raster = urbanGriddedPop(normalized_world_pop)
urban_extents = urban_raster.calculateUrban(densVal=300, totalPopThresh=5000, smooth=True)
wp_urbanExtents = r"UBZ_urban_clusters_world_pop_normalized_smooth.shp"
urban_extents.to_file(wp_urbanExtents)

### run ghs_pop urban centres


In [34]:
urban_raster = urbanGriddedPop(ghs_pop_ubz_file)
urban_extents = urban_raster.calculateUrban(densVal=1500, totalPopThresh=50000, smooth=True)
wp_urbanExtents = r"UBZ_urban_centres_ghs_pop_smooth.shp"
urban_extents.to_file(wp_urbanExtents)

### run ghs_pop urban clusters

In [35]:
urban_raster = urbanGriddedPop(ghs_pop_ubz_file)
urban_extents = urban_raster.calculateUrban(densVal=300, totalPopThresh=5000, smooth=True)
wp_urbanExtents = r"UBZ_urban_clusters_ghs_pop_smooth.shp"
urban_extents.to_file(wp_urbanExtents)